<div class="alert alert-block alert-success">
    
## Python libraries used by this module
    
</div>

In [1]:
%matplotlib inline
from sys import path as spath
spath.append("scripts/") #Location of the modules created for this pipeline

#Modules created for this pipeline
from CRISPRtools import * #MakeFasta, PilerCRReader, MinCEDReader
from easyFunctions import BLAST_short, Coordinate, dump
from InfernalResults import *
from HMMParser import *
from Rho import *

#Native and conda installed modules
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.SeqIO import index as fasta_index, parse, write
from Bio.SeqRecord import SeqRecord
from os import chdir, path, stat, system
from pandas import Series
from pickle import load
from RNA import fold as fold_rna
import IPython.display as ipd
#done = ipd.Audio('DoneSound.wav',autoplay=True)
print("Loaded libraries")

#done

Loaded libraries


<div class="alert alert-block alert-danger">
    
## Load the assembly pseudochromosomes that have a CRISPR array and a Cas9
    
</div>

In [38]:
chdir("/home/ec2-user/TRACR_RNA/data") # path to the data folder from the git repo
allAssemblies = dict(fasta_index("proteins/test_Cas12.faa",'fasta')) 
protIDs = allAssemblies.keys()
erpEXE_Path = "../scripts/Arnold/erpin"
rhoDatabase = "../scripts/Arnold/rho-indep.epn"
casOperons = load(open("pickles/Cas12_operons.p","rb"))

<div class="alert alert-block alert-success">
    
## Search through each pseudochromosome
    
</div>

In [39]:
noPredictedTracr = {} # {ID:Reason it didn't have a tracr}
totalSols, erpSols, breakCount, hadToGetSeq = 0, 0, 0, 0
sgFolds = open("tables/AllPredictedSgRNA_folds.tsv","w")
sgFolds.write("\t".join(["Seq Name","Repeat Folding Count","Repeat Length","tracr strand","tracrSeq","sgRNA","Fold","Consensus Repeat","Repeat Dir"])+'\n')
sgRNASols = open("sequences/AllPredictedSgRNAs.fasta","w")
possibleSol = open("sequences/AllPredicted_TracrRNAs.fasta","w")
breakPoints = set(range(0,len(allAssemblies),100))

print("Searching through %i assemblies for tracrRNAs.\n" % (len(allAssemblies)))
for i, protID in enumerate(allAssemblies):
    if i in breakPoints: print(i,end=' ')

    # Step 1. Get the CRISPR 
    operon = casOperons[protID]
    crispr = operon.getCRISPR(protID)

    # Step 2. Write all consensus repeats to a file
    if not path.exists(operon.getRepeatPath(protID)): crispr.repeatSeqs(protID,open(operon.getRepeatPath(protID),'w'))

    # Step 3. Blast the consensus repeats against the Cas-like protein-containing chromosome
    try:
        if not path.exists("blastout/conRepeats/%s.xml" % (protID)):
            blast_output = BLAST_short(operon.getRepeatPath(protID), operon.getFastaPointer(protID), "blastout/conRepeats/%s.xml" % (protID))
            blastResults = parseBLAST(blast_output)
        blastResults = parseBLAST(NCBIXML.parse(open("blastout/conRepeats/%s.xml" % (protID),'r')))
    except:
        print('\nBlast error for '+protID,operon.getFastaPointer(protID),operon.getRepeatPath(protID))
        blastResults = []
    if len(blastResults) == 0: 
        noPredictedTracr[protID] = 'No BLAST results'
        continue

    #Step 4. Narrow the blast results down by removing crRNAs
    crispr.clusterBLASTResults(blastResults,protID)
     
    # Step 5. Get the approriate flanking sequence for each anti-repeat candidate
    if len(crispr.antiRepeats) == 0:
        noPredictedTracr[protID] = 'No Anti-repeats'
        continue
    antiPath = "sequences/antiCandidates/%s.fasta" % (protID)
    if not path.exists(antiPath):
        crispr.getAntiRepeatCandidates(open(antiPath,"w"), operon.getSeq())
        strSeq = operon.getSeq()
        with open(antiPath,"w") as fh:
            for anti in crispr.antiRepeats.values():
                anti.name =protID
                fh.write(anti.getSeq(strSeq)+'\n')

    # Step 6. Look for termination signals
    terminalPath = "sequences/rhoTerms/%s.out" % (protID)
    if not path.exists(terminalPath):
        res = system("%s %s %s -1,4 -add 2 4 2 -pcw 3.0 -cutoff 100%% > %s" % (erpEXE_Path, rhoDatabase, antiPath, terminalPath))
        if res != 0:
            noPredictedTracr[protID] = 'Erpin Failed %i' % (res)
            continue
    
    # Step 7. Read the termination signals
    erpOut = ErpinOut(outfile=terminalPath,inputfile=antiPath)
    erpSols += len(erpOut.terminators)

    # Step 8. Get tracrRNA candidates with rho-ind signals
    print(crispr.name)
    numNewTracrs = crispr.getTracrRNA_Candidates(erpOut,possibleSol,sgRNASols,sgFolds)
    if numNewTracrs == 0: noPredictedTracr[protID] = 'No terminators'
    
    # Keep track of how many solutions have been found so far and print
    totalSols += numNewTracrs

possibleSol.close()
sgFolds.close()
sgRNASols.close()
print("\nErpin Solutions:", erpSols)
print("Found %i possible tracr solutions from %i assmeblies" % (totalSols,i-len(noPredictedTracr)))
#done

Searching through 1 assemblies for tracrRNAs.

0 [SeqRecord(seq=Seq('TTTTTGCGATTTCGGAATTTTCGCCCGATA', SingleLetterAlphabet()), id='3300008077.a:Ga0105964_1000553_1:1', name='3300008077.a:Ga0105964_1000553_1:1', description='3300008077.a:Ga0105964_1000553_1:1', dbxrefs=[]), SeqRecord(seq=Seq('TACGTTGATGATTGGGACGCACAAAATCGG', SingleLetterAlphabet()), id='3300008077.a:Ga0105964_1000553_1:2', name='3300008077.a:Ga0105964_1000553_1:2', description='3300008077.a:Ga0105964_1000553_1:2', dbxrefs=[]), SeqRecord(seq=Seq('TGTGTATGATACTGGAGATAATATGCAGA', SingleLetterAlphabet()), id='3300008077.a:Ga0105964_1000553_1:3', name='3300008077.a:Ga0105964_1000553_1:3', description='3300008077.a:Ga0105964_1000553_1:3', dbxrefs=[]), SeqRecord(seq=Seq('TGATATGTATTTCAATGTCGCTTGGCATAA', SingleLetterAlphabet()), id='3300008077.a:Ga0105964_1000553_1:4', name='3300008077.a:Ga0105964_1000553_1:4', description='3300008077.a:Ga0105964_1000553_1:4', dbxrefs=[]), SeqRecord(seq=Seq('CCATGAAAGACGAGGCATTTAAGAATACA', Sing

<div class="alert alert-block alert-success">
    
<h2><a id="round0Prep">Profile predicted tracrRNA results</a></h2>

#### Cluster tracrRNA predictions
#### Q1. How many systems have a possible solution?
#### Q2. How many systems have only 1?
    
</div>

In [40]:
%%bash 
cd-hit-est -i sequences/AllPredicted_TracrRNAs.fasta -o sequences/AllPredictedTracrRNAs.grouped.fasta -T 0 -M 0 -d 0 -c .95 -s .9 -sc 1 >logs/Cas9_tracrClusterLog.log
tail -n 8 logs/Cas9_tracrClusterLog.log > logs/clusterInfo
head -n 1 logs/clusterInfo; rm logs/clusterInfo
rm sequences/AllPredictedTracrRNAs.grouped.fasta
mv sequences/AllPredictedTracrRNAs.grouped.fasta.clstr clusters/

        3  finished          3  clusters


In [35]:
def count_members(seqIDList):
    seqCounter = Counter()
    for seqID in seqIDList: seqCounter[base_id(seqID)]+=1
    return seqCounter

def base_id(seqID): return seqID[:seqID.rfind("_")]

In [44]:
allPredictedTracrs = fasta_index("sequences/AllPredicted_TracrRNAs.fasta","fasta")
allSgRNA_Sols = fasta_index("sequences/AllPredictedSgRNAs.fasta","fasta")
clusterInfo = processClusterFile("clusters/AllPredictedTracrRNAs.grouped.fasta.clstr")
dist={}
allSeqCounter = Counter()
for cluster in clusterInfo: 
    sub = count_members(clusterInfo[cluster].members)
    dist[cluster]={"Total":len(sub),"Sequences":len(clusterInfo[cluster].members)} 
    for member,count in sub.most_common(len(sub)): allSeqCounter[member]+=count
singleMaps = set()
for member,count in allSeqCounter.most_common(len(allSeqCounter)): 
    if count==1: singleMaps.add(member)
print("A1. There are %s systems with 1 or more potential TracrRNA out of %s systems"% (comma(len(allSeqCounter)),comma(len(allAssemblies))))
print("A2. There are %s systems that have only 1 predicted BlastHit+Rho sequence" % comma(len(singleMaps)))

removedClusters,singleClusters=set(),set()
removedPossibles=0
for cluster in clusterInfo:
    clMembers = count_members(clusterInfo[cluster].members)
    if len(clMembers) < 5:
        if len(singleMaps.intersection(clMembers)) == 0:
            removedClusters.add(cluster)
            for member,count in clMembers.most_common(len(clMembers)):
                curCount = allSeqCounter[member]
                allSeqCounter[member]-=count
                removedPossibles += int(allSeqCounter[member]==0)
                assert(allSeqCounter[member]>=0)
            continue
        else: singleClusters.add(cluster)
    cluster = cluster.replace(" ","_")

    with open("conseqs0/%s.fasta" %(cluster),'w') as fh:
        seqCounter = Counter()
        for sID in clusterInfo[cluster.replace("_"," ")].members:
            seq = str(allPredictedTracrs[sID].seq).upper()
            seqCounter[seq] +=1
            if seqCounter[seq]>2: continue #Don't write the same sequence to the file more than once
            write(allPredictedTracrs[sID],fh,"fasta")
            
print("Built %i Covariance models from %i Clusters.\n\tRemoved: %i\n\tWould have been removed but had singles: %i\n\tSequences that not longer have a potential Tracr: %i" % (len(clusterInfo)-len(removedClusters),len(clusterInfo),len(removedClusters),len(singleClusters),removedPossibles))

3300008077.a:Ga0105964_1000553_3
3
A1. There are 1 systems with 1 or more potential TracrRNA out of 1 systems
A2. There are 0 systems that have only 1 predicted BlastHit+Rho sequence
Built 0 Covariance models from 3 Clusters.
	Removed: 3
	Would have been removed but had singles: 0
	Sequences that not longer have a potential Tracr: 1


<div class="alert alert-block alert-success">
    
<h2>Build Covariance Models for clusters that:</h2>
<font color=blue>

1. contain a sequence from at least 5 other assemblies OR
2. has a sequence for a system that only has a single potential TracrRNA

</font>
</div>

<div class="alert alert-block alert-success">
    
<h2>Perform the HMMER Search</h2>

</div>


In [46]:
%%bash
python ../scripts/StructureSearchScriptBuilder.py 1

Running conseqs1


Traceback (most recent call last):
  File "../scripts/StructureSearchScriptBuilder.py", line 83, in <module>
    Main(int(argv[1]))
  File "../scripts/StructureSearchScriptBuilder.py", line 30, in Main
    chdir("/mnt/research/germs/shane/transActRNA/data/conseqs%i" % (runNum))
FileNotFoundError: [Errno 2] No such file or directory: '/mnt/research/germs/shane/transActRNA/data/conseqs1'


CalledProcessError: Command 'b'python ../scripts/StructureSearchScriptBuilder.py 1\n'' returned non-zero exit status 1.

<div class="alert alert-block alert-success">
    
<h2>Read the results from the hmm search</h2>

</div>

In [ ]:
infernalResults = ProcessInfernal(0,"Cas9")